In [70]:
import pandas as pd
import matplotlib.pyplot as plt
from tree_based_models import model_selection_using_kfold, get_model
from sklearn.impute import SimpleImputer

In [71]:
# Clinical Data
clinical_train = pd.read_csv("data/X_train/clinical_train.csv")
clinical_test = pd.read_csv("data/X_test/clinical_test.csv")

# Molecular Data
molecular_train = pd.read_csv("data/X_train/molecular_train.csv")
molecular_test = pd.read_csv("data/X_test/molecular_test.csv")

target_train = pd.read_csv("data/X_train/target_train.csv")

# Preview the data
clinical_train.head()

,ID,CENTER,BM_BLAST,WBC,ANC,MONOCYTES,HB,PLT,CYTOGENETICS
0,P132697,MSK,14.0,2.8,0.2,0.7,7.6,119.0,"46,xy,del(20)(q12)[2]/46,xy[18]"
1,P132698,MSK,1.0,7.4,2.4,0.1,11.6,42.0,"46,xx"
2,P116889,MSK,15.0,3.7,2.1,0.1,14.2,81.0,"46,xy,t(3;3)(q25;q27)[8]/46,xy[12]"
3,P132699,MSK,1.0,3.9,1.9,0.1,8.9,77.0,"46,xy,del(3)(q26q27)[15]/46,xy[5]"
4,P132700,MSK,6.0,128.0,9.7,0.9,11.1,195.0,"46,xx,t(3;9)(p13;q22)[10]/46,xx[10]"


## Data Preprocessing

In [72]:
train = pd.concat(
    [
        clinical_train.set_index("ID").select_dtypes(include="number"),
        target_train.set_index("ID"),
    ],
    axis=1,
)
train = train[~train["OS_YEARS"].isna()]
train.head()

,BM_BLAST,WBC,ANC,MONOCYTES,HB,PLT,OS_YEARS,OS_STATUS
ID,,,,,,,,
P132697,14.0,2.8,0.2,0.7,7.6,119.0,1.115068,1.0
P132698,1.0,7.4,2.4,0.1,11.6,42.0,4.928767,0.0
P116889,15.0,3.7,2.1,0.1,14.2,81.0,2.043836,0.0
P132699,1.0,3.9,1.9,0.1,8.9,77.0,2.476712,1.0
P132700,6.0,128.0,9.7,0.9,11.1,195.0,3.145205,0.0


In [73]:
tmp = molecular_train.groupby("ID").size().to_frame("Nmut")
train = train.merge(tmp, left_index=True, right_index=True, how="inner")

In [74]:
train.columns

Index(['BM_BLAST', 'WBC', 'ANC', 'MONOCYTES', 'HB', 'PLT', 'OS_YEARS',
       'OS_STATUS', 'Nmut'],
      dtype='object')

## Kfold

In [75]:
target = "OS_YEARS"
features = ["BM_BLAST", "ANC", "MONOCYTES", "HB", "PLT", "Nmut"]
model_type = "ridge_benchmark"

In [76]:
model_selection_using_kfold(
    data=train.reset_index(),
    target=target,
    features=features,
    model_type=model_type,
    feat_engineering=None,
    unique_id="ID",
    plot_ft_importance=False,
    n_splits=6,
    log=False,
)

Fold 1 - IPCW C-index (Train: 0.6829 | Test: 0.6696)
Fold 2 - IPCW C-index (Train: 0.6771 | Test: 0.6885)
Fold 3 - IPCW C-index (Train: 0.6758 | Test: 0.7011)
Fold 4 - IPCW C-index (Train: 0.6795 | Test: 0.6797)
Fold 5 - IPCW C-index (Train: 0.6846 | Test: 0.6628)
Fold 6 - IPCW C-index (Train: 0.6810 | Test: 0.6717)
IPCW C-index: 0.6789 (± 0.0128) [Min: 0.6628 ; Max: 0.7011]


## Eval

In [77]:
test = clinical_test

tmp = molecular_test.groupby("ID").size().to_frame("Nmut")
test = test.merge(tmp, on="ID", how="left").fillna({"Nmut": 0})
test

,ID,CENTER,BM_BLAST,WBC,ANC,MONOCYTES,HB,PLT,CYTOGENETICS,Nmut
0,KYW1,KYW,68.0,3.45,0.5865,NaN,7.6,48.0,"47,XY,+X,del(9)(q?)[15]/47,XY,+X[5]",4.0
1,KYW2,KYW,35.0,3.18,1.2402,NaN,10.0,32.0,"46,XY,der(3)?t(3;11)(q26.2;q23),add(4)(p15).de...",3.0
2,KYW3,KYW,NaN,12.40,8.6800,NaN,12.3,25.0,"47,XX,+8",3.0
3,KYW4,KYW,61.0,5.55,2.0535,NaN,8.0,44.0,Normal,3.0
4,KYW5,KYW,2.0,1.21,0.7381,NaN,8.6,27.0,"43,XY,dic(5;17)(q11.2;p11.2),-7,-13,-20,-22,+r...",3.0
...,...,...,...,...,...,...,...,...,...,...
1188,KYW1189,KYW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1189,KYW1190,KYW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
1190,KYW1191,KYW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1191,KYW1192,KYW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0


In [78]:
imputer = SimpleImputer(strategy="median")
X_train = imputer.fit_transform(train[features])
X_test = imputer.fit_transform(test[features])

model = get_model(model_type=model_type)
model.fit(X_train, train[target])
preds = -model.predict(X_test)

submission = pd.Series(preds, index=test["ID"], name="risk_score")
submission

ID
KYW1       0.231692
KYW2      -1.355000
KYW3      -2.638500
KYW4      -0.179906
KYW5      -2.016700
             ...   
KYW1189   -2.114031
KYW1190   -1.974888
KYW1191   -2.253173
KYW1192   -1.974888
KYW1193   -2.114031
Name: risk_score, Length: 1193, dtype: float64

In [79]:
submission.to_csv("data/base_submission.csv")

In [80]:
submission

ID
KYW1       0.231692
KYW2      -1.355000
KYW3      -2.638500
KYW4      -0.179906
KYW5      -2.016700
             ...   
KYW1189   -2.114031
KYW1190   -1.974888
KYW1191   -2.253173
KYW1192   -1.974888
KYW1193   -2.114031
Name: risk_score, Length: 1193, dtype: float64

In [81]:
len(
    set(pd.read_csv("data/benchmark_submission.csv")["ID"]) & set(submission.index)
) / len(set(submission.index))

1.0